In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

from coeditor.dataset import TokenizedEditDataset
from coeditor.model import *
from coeditor.encoding import TokenizedEdit, decode_tokens, tokens_to_change
import shutil
import random
from prepare_data import make_or_load_datasets

os.chdir(proj_root())

In [2]:
# test_data_name = "medium"
test_data_name = "SPOT"
encoder = AnalysisBasedEditEncoder(extra_ctx_names=("usees", "post-usees"))
datasets = make_or_load_datasets(test_data_name, encoder, recreate_data=False)
test_data = datasets["test"]

In [3]:
rep_edits = pfilter(TokenizedEdit.is_repetitive_edit, test_data.all_edits())
small_edits = pfilter(TokenizedEdit.is_small_edit, test_data.all_edits())
refactor_edits = [e for e in test_data.all_edits() if e.updated_calls]
print("Total edits: ", len(list(test_data.all_edits())))
print("Repetitive edits: ", len(rep_edits))
print("Small edits: ", len(small_edits))
print("Refactoring edits: ", len(refactor_edits))

filtering: 100%|██████████| 1395/1395 [00:00<00:00, 4041.43it/s]

Total edits:  1395
Repetitive edits:  75
Small edits:  805
Refactoring edits:  182


In [4]:
# model_dir = get_model_dir() / "coeditor-medium-file"
model_dir = get_model_dir() / "coeditor-medium-analysis-post_usees"
model = CoeditorModel.load_pretrained(model_dir)
model.to("cuda:1")

eval_args = EvalArgs(4096 * 2)
dec_args = DecodingArgs(num_beams=1)

In [5]:
eval_dir = model_dir / "evals" / test_data_name
eval_cache = PickleCache(eval_dir)

In [16]:
model.data_args.use_signature_prefix = False
refactor_data = TokenizedEditDataset.from_edits(refactor_edits)
refactor_result = model.predict_on_data(refactor_data, eval_args, dec_args)
call_acc, call_correct = refactor_result.call_update_accuracy()
print("Call update accuracy: ", call_acc)

decoding: 100%|██████████| 38/38 [02:01<00:00,  3.20s/batch]


20 / 200 calls were considered incorrect since they failed to parse.
Call update accuracy:  (mean=0.31, weight=200)


In [14]:
model.data_args.use_signature_prefix = True
refactor_data = TokenizedEditDataset.from_edits(refactor_edits)
eval_args = eval_args = EvalArgs(4096 * 4)
refactor_result = model.predict_on_data(refactor_data, eval_args, dec_args)
call_acc, call_correct = refactor_result.call_update_accuracy()
print("(use_signature_prefix) Call update accuracy: ", call_acc)

decoding: 100%|██████████| 38/38 [02:33<00:00,  4.05s/batch]


18 / 200 calls were considered incorrect since they failed to parse.
(use_signature_prefix) Call update accuracy:  (mean=0.39, weight=200)


In [11]:
out_dir = eval_dir / "CallUpdateAccuracy.txt"
refactor_result.save_examples_to_dir(out_dir, call_correct)
print(out_dir)

saving examples: 100%|██████████| 182/182 [00:01<00:00, 92.27it/s] 

/mnt/nas/jiayi/coeditor/models/trained/coeditor-medium-analysis-post_usees/evals/SPOT/CallUpdateAccuracy.txt


In [9]:
rep_data = TokenizedEditDataset.from_edits(rep_edits)
rep_result = model.predict_on_data(rep_data, eval_args, dec_args)
display(rep_result.exact_match_accuracy()[0])

(mean=0.74667, weight=75)

In [7]:
dec_args = DecodingArgs(num_beams=1)
dec_result = eval_cache.cached("DatasetDecodingResult.pkl", lambda: model.predict_on_data(test_data, eval_args, dec_args))